In [1]:
import pandas as pd
import json
import os
from glob import glob
from pathlib import Path
import cv2
import csv
import re
from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt

In [2]:
path = "walt_annotations/cmu_data/first_batch.json"
file = open(path)

anns = json.load(file)
anns_info = anns['info']
anns_categories = anns['categories']
anns_licenses = anns['licenses']
anns_images = anns['images']
anns_annotations = anns['annotations']

FileNotFoundError: [Errno 2] No such file or directory: 'walt_annotations/cmu_data/first_batch.json'

In [23]:
def load_img(path):
    img = cv2.imread(path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    return img

img_path = "/home/bstandaert/Documents/KDworkshop/WALT-tests/bank/images/2021-05-01T00-02-58.359379.jpg"
img = load_img(img_path)
img = cv2.resize(img, (640, 480), interpolation = cv2.INTER_AREA)

img.shape

(480, 640, 3)

In [5]:
folder_path = "PATHTO/WALT-challenge/cam1/test"

images = sorted(glob(os.path.join(folder_path, "images/*.jpg")))
annotations = sorted(glob(os.path.join(folder_path, "labelsFOLDER/*.txt")))

output_vid = os.path.join(folder_path, "VIDEONAME.mp4")
fps = 2


def load_img(path):
    img = cv2.imread(path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    return img

def load_bboxes(path):
    bboxes = []
    with open(path) as f:
        for line in f.readlines():
            bbox = [int(s) for s in re.findall(r'\b\d+\b', line)]
            bboxes.append(bbox)
    return np.asarray(bboxes)

def load_bboxes2(path):
    bboxes = []
    with open(path) as f:
        for line in f.readlines():
            bbox = line.split(' ')
            bbox = np.asarray([float(bbox[1]), float(bbox[2]), float(bbox[3]), float(bbox[4])])
            bboxes.append(bbox)
    return np.asarray(bboxes)

def draw_bboxes(img, bboxes):
    for bbox in bboxes:
        img = cv2.rectangle(
            img,
            (int(bbox[0]), int(bbox[1])),
            (int(bbox[0]), int(bbox[1])),
            color=[255, 0, 0],
            thickness=3,
            lineType=cv2.LINE_AA,
        )
    return img

def draw_bboxes2(img, bboxes):
    h, w = img.shape[:2]
    for bbox in bboxes:
        tl = (int(w * (bbox[0]-bbox[2]/2)), int(h * (bbox[1]-bbox[3]/2)))
        br = (int(w * (bbox[0]+bbox[2]/2)), int(h * (bbox[1]+bbox[3]/2)))
        img = cv2.rectangle(
            img,
            tl,
            br,
            color=[255, 0, 0],
            thickness=1,
            lineType=cv2.LINE_AA,
        )
    return img

def final_img(img):
    return cv2.cvtColor(img, cv2.COLOR_RGB2BGR).astype(np.uint8)

def print_filename(img, annotation):
    name = os.path.basename(annotation)
    h = img.shape[0]
    cv2.putText(
        img,
        name,
        (50, h - 50),
        fontFace=0,
        fontScale=1,
        thickness=1,
        color=[255, 0, 0],
        lineType=cv2.LINE_AA,
    )
    return img

img = load_img(images[0])


resize=False

if resize:
    video_writer = cv2.VideoWriter(
        output_vid,
        cv2.VideoWriter_fourcc(*"mp4v"),
        float(fps),
        (640, 480),
    )
else:
    video_writer = cv2.VideoWriter(
        output_vid,
        cv2.VideoWriter_fourcc(*"mp4v"),
        float(fps),
        (img.shape[1], img.shape[0]),
    )


for image, annotation in tqdm(zip(images, annotations), total=len(images)):
    img = load_img(image)
    if resize:
        img = cv2.resize(img, (640, 480), interpolation = cv2.INTER_AREA)
    
    bboxes = load_bboxes2(annotation)
    img = print_filename(img, image)
    img = draw_bboxes2(img, bboxes)
    img = final_img(img)
    video_writer.write(img)

video_writer.release()

100%|█████████████████████████████████████████| 966/966 [02:49<00:00,  5.72it/s]


In [4]:
import pandas as pd
import json
import os
from glob import glob
from pathlib import Path
import cv2
import csv
import re
from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
def load_img(path):
    img = cv2.imread(path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    return img

def load_new_bboxes(path):
    bboxes = []
    with open(path) as f:
        for line in f.readlines():
            bbox = [float(s) for s in line.split(' ')]
            bboxes.append(bbox[1:])
    return np.asarray(bboxes)

def plot_new_img(img, new_bboxes):
    h, w = img.shape[:2]
    new_bboxes[:, 0::2] *= w
    new_bboxes[:, 1::2] *= h
    for bbox in new_bboxes:
        img = cv2.rectangle(
            img,
            (int(bbox[0] - bbox[2]/2), int(bbox[1] - bbox[3]/2)),
            (int(bbox[0] + bbox[2]/2), int(bbox[1] + bbox[3]/2)),
            color=[255, 0, 0],
            thickness=5,
            lineType=cv2.LINE_AA,
        )
    plt.imshow(img)
    
def load_old_bboxes(path):
    old_bboxes = []
    with open(path) as f:
        for line in f.readlines():
            bbox = [int(s) for s in re.findall(r'\b\d+\b', line)]
            old_bboxes.append(bbox)
    return np.asarray(old_bboxes)

def transform_bboxes(img, old_bboxes):
    h, w = img.shape[:2]
    centers_x = (old_bboxes[:, 2]/2 + old_bboxes[:, 0])
    centers_y = (old_bboxes[:, 3]/2 + old_bboxes[:, 1])
    widths = old_bboxes[:, 2]
    heights = old_bboxes[:, 3]
    new_bboxes = np.stack([centers_x, centers_y, widths, heights], axis=1)
    new_bboxes[:, 0::2] /= w
    new_bboxes[:, 1::2] /= h
    return new_bboxes

def new_bboxes_str(new_bboxes):
    out = ""
    for bbox in new_bboxes:
        out += f"0 {bbox[0]} {bbox[1]} {bbox[2]} {bbox[3]}\n"
    return out

In [ ]:
new_png = "/home/bstandaert/Documents/KDworkshop/AI-city/S05c016/bank/images/frame_0000.png"
new_txt = "/home/bstandaert/Documents/KDworkshop/AI-city/S05c016/bank/labels/frame_0000.txt"

new_img = load_img(new_png)
new_bboxes = load_new_bboxes(new_txt)
    
plot_new_img(new_img, new_bboxes)

In [ ]:
old_jpg = "tests/254_1550879441.jpg"
old_txt = "tests/254_1550879441.txt"
    
old_img = load_img(old_jpg)
old_bboxes = load_old_bboxes(old_txt)

new_bboxes = transform_bboxes(old_img, old_bboxes)

plot_new_img(old_img, new_bboxes)

In [ ]:
path_images = "/media/bstandaert/SSD-2/WALT-challenge/cam3/test/images"
path_labels = "/media/bstandaert/SSD-2/WALT-challenge/cam3/test/labels"
path_new_labels = "/media/bstandaert/SSD-2/WALT-challenge/cam3/test/new_labels"

images = sorted(glob(os.path.join(path_images, "*.jpg")))
old_labels = sorted(glob(os.path.join(path_labels, "*.txt")))

for image, old_label in tqdm(zip(images, old_labels), total=len(images)):
    img = load_img(image)
    old_bboxes = load_old_bboxes(old_label)
    new_bboxes = transform_bboxes(img, old_bboxes)
    out = new_bboxes_str(new_bboxes)
    
    new_file = os.path.join(path_new_labels, os.path.basename(old_label))
    with open(new_file, "x") as f:
        f.write(out)

In [ ]:
folder_path = ""

images = sorted(glob(os.path.join(folder_path, "images/*.jpg")))
annotations = sorted(glob(os.path.join(folder_path, "gt/*.txt")))

for image, label in tqdm(zip(images, labels), total=len(images)):
    
    img = load_img(image)
    bboxes = load_bboxes()
    
    img = draw_bboxes(img, bboxes)